In [1]:
from tinyec import registry
import secrets
import time
import paho.mqtt.client as mqtt

def initApp():
    global BROKER, PORT, ENCRYPTED_TEXT_TOPIC, CLIENT_PUBLIC_KEY_TOPIC, CLOUD_PUBLIC_KEY_TOPIC, client
    BROKER = "localhost"
    PORT = 1883
    CLIENT_PUBLIC_KEY_TOPIC = "client_pubkey"
    CLOUD_PUBLIC_KEY_TOPIC = "cloud_pubkey"
    ENCRYPTED_TEXT_TOPIC = "encrypted_txt"
    client = mqtt.Client("localhost", "792792")
    client.connect(BROKER, PORT)
    

curve = registry.get_curve('brainpoolP256r1')

def compress_point(point):
    return hex(point.x) + hex(point.y % 2)[2:]


#--------------------------------------------------------------------
#   Generate CLOUD_PRIVATE_KEY and CLOUD_PUBLIC_KEY

cloud_privKey = secrets.randbelow(curve.field.n)
cloud_pubKey = cloud_privKey * curve.g

#--------------------------------------------------------------------

def shared_key(privKey, other_pubKey):
    sharedECCKey = other_pubKey * privKey
    return sharedECCKey

def ecc_decrypt(encrypted, cloud_privKey, client_pubKey):
    # Retrieve the shared ECC decryption key
    decryptKey = shared_key(cloud_privKey, client_pubKey)

    decrypted = bytearray()
    shared_key_hex = compress_point(decryptKey)
    for byte in encrypted:
        decrypted_byte = byte ^ int(shared_key_hex, 16) % 256  # Limit to byte range
        decrypted.append(decrypted_byte)

    # Convert the decrypted bytes back to string
    decrypted_message = decrypted.decode()
    return decrypted_message

#--------------------------------------------------------------------
#   Connect to an MQTT brocker and publish a topic
def mqttCommunicateDataTask(cloud_pubKey):
    client.publish(CLOUD_PUBLIC_KEY_TOPIC, str(cloud_pubKey))


#--------------------------------------------------------------------
#   Connect to an MQTT brocker and subscribe to a topic
def mqttReceiveDataTask():
    client_pubKey = client.subscribe(CLIENT_PUBLIC_KEY_TOPIC)
    encrypted = client.subscribe(ENCRYPTED_TEXT_TOPIC)
    return (client_pubKey,encrypted)


#--------------------------------------------------------------------
#   Here you can generate your file


#   Decrypt the file using ECC decryption

(client_pubKey,encrypted) =mqttReceiveDataTask()
decrypted_message = ecc_decrypt(encrypted, cloud_privKey, client_pubKey)
print("Decrypted:", decrypted_message)

#   Send the file through MQTT to publish data
def loopApp():
    while True:
        
        mqttCommunicateDataTask(cloud_pubKey)
        (client_pubKey,encrypted) =mqttReceiveDataTask()
        decrypted_message = ecc_decrypt(encrypted, cloud_privKey, client_pubKey)
        print("Decrypted:", decrypted_message)
        time.sleep(5)

if __name__ == '__main__':
    initApp()
    try: 
        loopApp()
    except:
        KeyboardInterrupt


#   END OF CODE



NameError: name 'client' is not defined